## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2021-12-01 14:57:47--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2021-12-01 14:57:48--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc19cdb602ae476569c239e18298.dl.dropboxusercontent.com/cd/0/inline/BbDgCqVTzlqOFjPi01qwdOTb_wKSPc-hvJfSn2GngqzlZ8Jg3b3uGqRQJb1ws6M4jn_jjtnGATwjKyXgwOVIW_oAwJ5azwrbWrFBhEvIeov-dvQOeKDRCiKre2BBHKYQRdHn3XEekDwW7qDc1O4Gno9Y/file# [following]
--2021-12-01 14:57:48--  https://uc19cdb602ae476569c239e18298.dl.dropboxusercontent.com/cd/0/inline/BbDgCqVTzlqOFjPi01qwdOTb_wKSPc-hvJfSn2GngqzlZ8Jg3b3uGqRQJb1ws6M4jn_jjtnGATwjKy

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [2]:
import pandas as pd
import numpy as np

#1
filename = '/home/alberto/data/data_000637.txt'
num_lines = sum(1 for line in open(filename))
N = np.random.randint(1e4, num_lines+1)
data = pd.read_csv(filename, nrows=N)
print(num_lines)
data

1310721


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
34281,1,0,85,3869200508,578,13
34282,1,0,58,3869200508,603,26
34283,1,0,139,3869200508,611,0
34284,1,0,56,3869200508,609,12


In [3]:
#2
data_group_orbit = data.groupby(['ORBIT_CNT'])['BX_COUNTER']
bx_cnts_max = data_group_orbit.max().max()+1
freq = sum(1 for item in data_group_orbit.max() if item == bx_cnts_max-1)
print("BX_COUNTS MAX:", bx_cnts_max)
print("Encountered", freq, "times")

BX_COUNTS MAX: 3564
Encountered 9 times


In [4]:
#3
data_orb_max = data[data['ORBIT_CNT']==data['ORBIT_CNT'].max()]
data_orb_bx_max = data_orb_max[data_orb_max['BX_COUNTER']==data_orb_max['BX_COUNTER'].max()]
time = ((data['ORBIT_CNT'].max() - data['ORBIT_CNT'].min())*bx_cnts_max + data_orb_max['BX_COUNTER'].max())*25 + data_orb_bx_max['TDC_MEAS'].max()*5/6
print("Data taken for", time, "ns")

Data taken for 30398384.166666668 ns


In [5]:
#4
data['TIME_ns'] = pd.Series( (data['ORBIT_CNT']*bx_cnts_max+data['BX_COUNTER'])*25 + data['TDC_MEAS']*5/6, index = data.index)
print("Last registered time:", data['TIME_ns'].values[-1])
data

Last registered time: 344745765278084.2


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
34281,1,0,85,3869200508,578,13,3.447458e+14
34282,1,0,58,3869200508,603,26,3.447458e+14
34283,1,0,139,3869200508,611,0,3.447458e+14
34284,1,0,56,3869200508,609,12,3.447458e+14


In [6]:
#5
data['HEAD'] = pd.Series(np.random.randint(2, size=data['HEAD'].size), index=data.index)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
1,0,0,124,3869200167,2374,27,3.447457e+14
2,0,0,63,3869200167,2553,28,3.447457e+14
3,0,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
34281,1,0,85,3869200508,578,13,3.447458e+14
34282,1,0,58,3869200508,603,26,3.447458e+14
34283,1,0,139,3869200508,611,0,3.447458e+14
34284,1,0,56,3869200508,609,12,3.447458e+14


In [7]:
#6
new_data = data[data['HEAD']==1].copy()
new_data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_ns
0,1,0,123,3869200167,2374,26,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
7,1,0,139,3869200167,2776,0,3.447457e+14
8,1,0,62,3869200167,2774,21,3.447457e+14
9,1,0,60,3869200167,2788,7,3.447457e+14
...,...,...,...,...,...,...,...
34281,1,0,85,3869200508,578,13,3.447458e+14
34282,1,0,58,3869200508,603,26,3.447458e+14
34283,1,0,139,3869200508,611,0,3.447458e+14
34284,1,0,56,3869200508,609,12,3.447458e+14


In [19]:
#7

fpga0 = new_data[new_data['FPGA']==0].groupby(['TDC_CHANNEL']).get_group()
fpga0

TypeError: get_group() missing 1 required positional argument: 'name'